<a href="https://colab.research.google.com/github/Tajalla2/LLMs-From-Scratch/blob/main/training_withGPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(torch.__version__)

2.3.0+cu121


In [2]:
print(torch.cuda.is_available())

True


In [3]:
tensor1 = torch.tensor([1.,2.,3.])
tensor2 = torch.tensor([4.,5.,6.])
print(tensor1+tensor2)

tensor([5., 7., 9.])


In [4]:
tensor1= tensor1.to("cuda")
tensor2=tensor2.to("cuda")
print(tensor1+tensor2)

tensor([5., 7., 9.], device='cuda:0')


In [5]:
tensor1= tensor1.to("cpu")
print(tensor1+tensor2)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

Single GPU training


In [6]:
xtrain = torch.tensor([
    [-1.2,3.1],
    [-0.9,2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]

])
ytrain = torch.tensor([0, 0, 0, 1, 1])
xtest = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])

ytest = torch.tensor([0, 1])

In [7]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
  def __init__(self,x,y):
    self.features=x
    self.labels=y

  def __getitem__(self,index):
    one_x = self.features[index]
    one_y = self.labels[index]
    return one_x,one_y

  def __len__(self):
    return self.labels.shape[0]

train_ds = ToyDataset(xtrain,ytrain)
test_ds = ToyDataset(xtest,ytest)

In [8]:
from torch.utils.data import DataLoader
torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=1,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=1
)

In [15]:
class NeuralNetwork(torch.nn.Module):
  def __init__(self,num_inputs,num_outputs):
    super().__init__()
    self.layers = torch.nn.Sequential(
        torch.nn.Linear(num_inputs,30),
        torch.nn.ReLU(),

        torch.nn.Linear(30,20),
        torch.nn.ReLU(),

        torch.nn.Linear(20,num_outputs),

    )
  def forward(self,x):
      logits = self.layers(x)
      return logits

In [16]:
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # NEW
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(),lr=0.5)
num_epochs=3

for epoch in range(num_epochs):
  model.train()
  for batch_idx, (features,labels) in enumerate(train_loader):
    features,labels = features.to(device), labels.to(device)
    logits= model(features)
    loss = F.cross_entropy(logits,labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    ##logging
    print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
          f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
          f" | Train/Val Loss {loss:.2f}")
  model.eval()

Epoch: 001/003 | Batch 000/002 | Train/Val Loss 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss 0.00


In [17]:
def compute_accuracy(model,dataloader,device):
  model = model.eval()
  correct = 0.0
  total_examples=0


  for idx, (features,labels) in enumerate(dataloader):
    features,labels = features.to(device),labels.to(device)

    with torch.no_grad():
      logits=model(features)

    predictions = torch.argmax(logits,dim=1)
    compare = labels == predictions
    correct+= torch.sum(compare)
    total_examples+=len(compare)

    return (correct/total_examples).item()

In [18]:
compute_accuracy(model,train_loader,device)

1.0

In [19]:
compute_accuracy(model, test_loader, device=device)

1.0